# Document Classification with Naive Bayes - Lab

## Introduction

In this lesson, you'll practice implementing the Naive Bayes algorithm on your own.

## Objectives

In this lab you will:  

* Implement document classification using Naive Bayes

## Import the dataset

To start, import the dataset stored in the text file `'SMSSpamCollection'`.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('SMSSpamCollection', sep='\t', names=['label', 'text'])
df.headd()

label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

## Account for class imbalance

To help your algorithm perform more accurately, subset the dataset so that the two classes are of equal size. To do this, keep all of the instances of the minority class (spam) and subset examples of the majority class (ham) to an equal number of examples.

In [2]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [45]:
ham = df[df['label']=='ham']
indices = np.random.choice(list(ham.index), 747, replace=False)
ham = ham.loc[indices]

In [46]:
ham.head()

label                                               text
2239   ham  Think you sent the text to the home phone. Tha...
2455   ham       Left dessert. U wan me 2 go suntec look 4 u?
4446   ham                      2 celebrate my bday, y else?
4950   ham                  I am in bus on the way to calicut
4502   ham                              So wat's da decision?

## Train-test split

Now implement a train-test split on the dataset: 

In [47]:
preprocessed = pd.concat([ham, df[df['label']=='spam']])

In [48]:
# Your code here
from sklearn.model_selection import train_test_split
X = preprocessed.drop('label', axis=1)
y = preprocessed['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

## Create the word frequency dictionary for each class

Create a word frequency dictionary for each class: 

In [49]:
# Your code here
classes = train_df['label'].unique()
word_freq_dict = {}
for cl in classes:
    bag = {}
    restrict = train_df[train_df['label']==cl]
    for row in restrict.index:
        doc = restrict['text'][row]
        for word in doc.split():
            bag[word] = bag.get(word, 0) + 1
    word_freq_dict[cl] = bag

## Count the total corpus words
Calculate V, the total number of words in the corpus: 

In [55]:
V = set()
for text in train_df['text']:
    for word in text.split():
        V.add(word)
        
V = len(V)
V

5893

## Create a bag of words function

Before implementing the entire Naive Bayes algorithm, create a helper function `bag_it()` to create a bag of words representation from a document's text.

In [56]:
def bag_it(text):
    bag = {}
    for word in text.split():
        bag[word] = bag.get(word, 0) + 1
    return bag

## Implementing Naive Bayes

Now, implement a master function to build a naive Bayes classifier. Be sure to use the logarithmic probabilities to avoid underflow.

In [57]:
# Your code here
def classify_doc(doc, class_word_freq, p_classes, V, return_posteriors=False):
    bag = bag_it(doc)
    classes = []
    posteriors = []
    for cl in class_word_freq.keys():
        p = np.log(p_classes[cl])
        for word in bag.keys():
            num = bag[word] + 1
            denom = class_word_freq[cl].get(word, 0) + V
            p += np.log(num/denom)
        classes.append(cl)
        posteriors.append(p)
    if return_posteriors:
        print(posteriors)
    return classes[np.argmax(posteriors)]

## Test your classifier

Finally, test your classifier and measure its accuracy. Don't be perturbed if your results are sub-par; industry use cases would require substantial additional preprocessing before implementing the algorithm in practice.

In [62]:
# Your code here
import numpy as np

p_classes = dict(preprocessed['label'].value_counts(normalize=True))

y_hat_train = X_train['text'].map(lambda x: classify_doc(x, word_freq_dict, p_classes, V))
resids = y_train == y_hat_train
resids.value_counts(normalize=True)

False    0.729464
True     0.270536
dtype: float64

## Level up (Optional)

Rework your code into an appropriate class structure so that you could easily implement the algorithm on any given dataset.

## Summary

Well done! In this lab, you practiced implementing Naive Bayes for document classification!